# In this File, every kernel are used to deal with one dataset

# Coding Format

## dataset name: 
## source:
## need to run additional first (maybe establish prompt or change label format)
## dataset discription: 

In [ ]:
# check the if folder "dataset" exist, if not establish one
# should login first
import os
import pandas as pd
target_directory = "dataset"
TRAIN_SPLIT = 0.65
TEST_SPLIT = 0.25 
VAL_SPLIT = 0.1
random_state = 42
HF_TOKEN = ""
# build directory
os.makedirs(target_directory, exist_ok=True)
# hf login 
!huggingface-cli login --token 

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `hf`CLI if you want to set the git credential as well.
Token is valid (permission: write).
The token `llama-3.2test` has been saved to C:\Users\User\.cache\huggingface\stored_tokens
Your token has been saved to C:\Users\User\.cache\huggingface\token
Login successful.
The current active token is: `llama-3.2test`


In [10]:
def size_checker(train:pd.DataFrame, test:pd.DataFrame, val:pd.DataFrame):
    print(
        f"The numeber of Train Dataset is: {train.shape[0]}\nThe numeber of Test Dataset is: {test.shape[0]}\nThe number of Val dataset is: {val.shape[0]}\n"
        )

In [11]:
## dataset name: "AI4Sec/cti-bench", "cti-mcq"
## source: huggingface
## need to run additional first (maybe establish prompt or change label format)
## dataset discription: 
### 
from datasets import load_dataset, DatasetDict

dataset_name = "cti-mcq"
dataset = load_dataset("AI4Sec/cti-bench", "cti-mcq")
# data are in the "test"
data = dataset['test']

train_tempdata = data.train_test_split(test_size=(1 - TRAIN_SPLIT) , seed = random_state)
train_dataset = train_tempdata['train']
temp_dataset = train_tempdata['test']

test_val_dataset = temp_dataset.train_test_split(test_size =  VAL_SPLIT / (VAL_SPLIT + TEST_SPLIT), seed=random_state)
test_dataset = test_val_dataset['train']
val_dataset = test_val_dataset['test']

os.makedirs(os.path.join(target_directory, dataset_name), exist_ok=True)
train_dataset = train_dataset.to_pandas()
test_dataset = test_dataset.to_pandas()
val_dataset = val_dataset.to_pandas()
train_dataset['label'] = train_dataset['GT']
test_dataset['label'] = test_dataset['GT']
val_dataset['label'] = val_dataset['GT']
train_dataset['prompt'] = train_dataset['Prompt']
test_dataset['prompt'] = test_dataset['Prompt']
val_dataset['prompt'] = val_dataset['Prompt']

train_dataset.to_csv(os.path.join(target_directory, dataset_name, "train.csv"), index=False)
test_dataset.to_csv(os.path.join(target_directory, dataset_name, "test.csv"), index=False)
val_dataset.to_csv(os.path.join(target_directory, dataset_name, "val.csv"), index=False)
size_checker(train_dataset, test_dataset, val_dataset)

The numeber of Train Dataset is: 1625
The numeber of Test Dataset is: 624
The number of Val dataset is: 251



In [12]:
## dataset name: Sentiment-Reasoning"
## source: huggingface
## need to run additional first (maybe establish prompt or change label format)
## dataset discription: 
### data do not contain prompt > make prompt for the dataset first than break them 
### use English to build the prompt, label as label
import pandas as pd
full_dataset = load_dataset("leduckhai/Sentiment-Reasoning")
from sklearn.model_selection import train_test_split
dataset_name = "sentiment-reasoning"
total_dataset = pd.concat([full_dataset['train'].to_pandas(), full_dataset['test'].to_pandas()])
prompt = """You are given a classification task from a Sentiment-Reasoning benchmark dataset. 
Your task is to choose the best option among the three categories provided. 
Return your answer as a single uppercase letter: E, F or G.

**Question:** {Question} 
**Options:** E) negative F) neutral G) positive 

**Important:** The last line of your answer should contain only one letter from the Options, with no additional text.
"""

total_dataset['prompt'] = total_dataset['English'].apply(lambda x: prompt.format(Question = x))

train_data, temp_data = train_test_split(total_dataset, test_size= (1-TRAIN_SPLIT),random_state=random_state)
test_data, val_data = train_test_split(temp_data, test_size =  VAL_SPLIT / (VAL_SPLIT + TEST_SPLIT), random_state=random_state)

os.makedirs(os.path.join(target_directory, dataset_name), exist_ok=True)

train_data.to_csv(os.path.join(target_directory, dataset_name, "train.csv"), index=False)
test_data.to_csv(os.path.join(target_directory, dataset_name, "test.csv"), index=False)
val_data.to_csv(os.path.join(target_directory, dataset_name, "val.csv"), index=False)
size_checker(train_data, test_data, val_data)

The numeber of Train Dataset is: 5066
The numeber of Test Dataset is: 1948
The number of Val dataset is: 780



In [ ]:
# ## dataset name: allenai/art
# ## source: huggingface
# ## dataset discription: 
# ### data do not contain prompt > make prompt for the dataset first than break them 
# from datasets import load_dataset
# import pandas as pd
# dataset_name = "allenai-art"
# full_dataset = load_dataset("allenai/art")

# prompt = """You are given a task from an Abductive Reasoning benchmark dataset. 
# Your task is to read the story and choose the most plausible hypothesis that explains it. 
# Return your answer as simple words: HA or HB.

# **Story:** {observation_1} {observation_2}
# **Options:** 
# HA) {hypothesis_1} HB) {hypothesis_2}

# **Important:** The last line of your answer should contain only HA or HB, with no additional text.
# """
# total_dataset = pd.concat([full_dataset['train'].to_pandas(), full_dataset['validation'].to_pandas()])
# def make_prompt(element):
#     return prompt.format(
#         observation_1 = element['observation_1'],
#         observation_2 = element['observation_2'],
#         hypothesis_1 = element['hypothesis_1'],
#         hypothesis_2 = element['hypothesis_2']
#         )
# total_dataset['prompt'] = total_dataset.apply(make_prompt, axis = 1)
# total_dataset['label'] = total_dataset['label'].apply(lambda x: {'1':'HA', '2':'HB'}[str(x)])
# train_data, temp_data = train_test_split(total_dataset, test_size= (1-TRAIN_SPLIT),random_state=random_state)
# test_data, val_data = train_test_split(temp_data, test_size =  VAL_SPLIT / (VAL_SPLIT + TEST_SPLIT), random_state=random_state)

# os.makedirs(os.path.join(target_directory, dataset_name), exist_ok=True)

# train_data.to_csv(os.path.join(target_directory, dataset_name, "train.csv"), index=False)
# test_data.to_csv(os.path.join(target_directory, dataset_name, "test.csv"), index=False)
# val_data.to_csv(os.path.join(target_directory, dataset_name, "val.csv"), index=False)
# size_checker(train_data, test_data, val_data)

The numeber of Train Dataset is: 111270
The numeber of Test Dataset is: 42797
The number of Val dataset is: 17119



In [ ]:
# skip for now, since this dataset is too big and overlap with the other one
# ## dataset name: openlifescienceai/medmcqa > osai-medmcqa
# ## source: huggingface
# ## dataset discription: 
# ### data do not contain prompt > make prompt for the dataset first than break them 
# from datasets import load_dataset
# import pandas as pd
# dataset_name = "osai-medmcqa"
# full_dataset = load_dataset("openlifescienceai/medmcqa")
# total_dataset = pd.concat([full_dataset['train'].to_pandas(), full_dataset['validation'].to_pandas(), full_dataset['test'].to_pandas()])
# total_dataset = total_dataset[total_dataset['cop'] != -1]
# prompt = """
# You are given a medical multiple-choice question from MedMCQA.
# Your task is to read the question and choose the most plausible answer. 
# Return your answer as simple words(OPA, OPB, OPC, or OPD).
# **Question:** {question}
# **Options:**
# OPA) {opa}
# OPB) {opb}
# OPC) {opc}
# OPD) {opd}
# **Important:** The last line of your answer should contain only OPA, OPB, OPC, or OPD, with no additional text.
# """
# def make_prompt(element):
#     return prompt.format(
#         question = element['question'],
#         opa = element['opa'],
#         opb = element['opb'],
#         opc = element['opc'],
#         opd = element['opd']
#     )
# total_dataset['prompt'] = total_dataset.apply(make_prompt, axis = 1)
# num2option = {
#     '0':'OPA',
#     '1':'OPB',
#     '2':'OPC',
#     '3':'OPD'
# }
# total_dataset['label'] = total_dataset['cop'].apply(lambda x : num2option[str(x)])
# train_data, temp_data = train_test_split(total_dataset, test_size= (1-TRAIN_SPLIT),random_state=random_state)
# test_data, val_data = train_test_split(temp_data, test_size =  VAL_SPLIT / (VAL_SPLIT + TEST_SPLIT), random_state=random_state)

# os.makedirs(os.path.join(target_directory, dataset_name), exist_ok=True)

# train_data.to_csv(os.path.join(target_directory, dataset_name, "train.csv"), index=False)
# test_data.to_csv(os.path.join(target_directory, dataset_name, "test.csv"), index=False)
# val_data.to_csv(os.path.join(target_directory, dataset_name, "val.csv"), index=False)
# size_checker(train_data, test_data, val_data)

The numeber of Train Dataset is: 121553
The numeber of Test Dataset is: 46751
The number of Val dataset is: 18701



In [15]:
## dataset name: textmachinelab/quail > tmlab-quail
## source: huggingface
## dataset discription: 
### data do not contain prompt > make prompt for the dataset first than break them 
from datasets import load_dataset
dataset_name = "tmlab-quail"
full_dataset = load_dataset("textmachinelab/quail")
total_dataset = pd.concat([full_dataset['train'].to_pandas(), full_dataset['validation'].to_pandas()])
prompt = """
You are given a medical multiple-choice question from quail.
Your task is to read the Context and choose the most plausible answer to the Question.
Return your answer as simple words(OP1, OP2, OP3, or OP4).
**Context:** {context}
**Question:** {question}
**Options:**
OP1) {OP1} 
OP2) {OP2}
OP3) {OP3}
op4) {OP4}
**Important:** The last line of your answer should contain only OP1, OP2, OP3, or OP4, with no additional text.
"""
num2answer = {
    '0':'OP1',
    '1':'OP2',
    '2':'OP3',
    '3':'OP4'
}
def make_prompt(element):
    answer = list(element['answers'])
    return prompt.format(
        context = element['context'],
        question = element['question'],
        OP1 = answer[0],
        OP2 = answer[1],
        OP3 = answer[2],
        OP4 = answer[3],
    )
total_dataset['prompt'] = total_dataset.apply(make_prompt, axis = 1)
total_dataset['label'] = total_dataset['correct_answer_id'].apply(lambda x: num2answer[str(x)])
train_data, temp_data = train_test_split(total_dataset, test_size= (1-TRAIN_SPLIT),random_state=random_state)
test_data, val_data = train_test_split(temp_data, test_size =  VAL_SPLIT / (VAL_SPLIT + TEST_SPLIT), random_state=random_state)

os.makedirs(os.path.join(target_directory, dataset_name), exist_ok=True)

train_data.to_csv(os.path.join(target_directory, dataset_name, "train.csv"), index=False)
test_data.to_csv(os.path.join(target_directory, dataset_name, "test.csv"), index=False)
val_data.to_csv(os.path.join(target_directory, dataset_name, "val.csv"), index=False)
size_checker(train_data, test_data, val_data)

The numeber of Train Dataset is: 8066
The numeber of Test Dataset is: 3102
The number of Val dataset is: 1242



In [16]:
## dataset name: openai/gsm8k-main > openai-gsm8k
## source: huggingface
## dataset discription: 
### data do not contain prompt > make prompt for the dataset first than break them 
from datasets import load_dataset
dataset_name = "openai-gsm8k"
full_dataset = load_dataset("openai/gsm8k", "main")
total_dataset = full_dataset['train'].to_pandas()
prompt = """
You are given a math question from gsm8k dataset.
Your task is to read the Context and answer the math question.
Just give the short answer as the response, try as simple as possible.
**Question:** {question}
**Important:** The last line of your answer should contain only the answer to the question, with no further explaination.
"""
def make_prompt(element):
    return prompt.format(
        question = element['question']
    )
def dealingwithanswer(element):
    position = element.find('####')
    return element[position + len('####'):]
total_dataset['prompt'] = total_dataset.apply(make_prompt, axis=1)
total_dataset['label'] = total_dataset['answer'].apply(dealingwithanswer)
train_data, temp_data = train_test_split(total_dataset, test_size= (1-TRAIN_SPLIT),random_state=random_state)
test_data, val_data = train_test_split(temp_data, test_size =  VAL_SPLIT / (VAL_SPLIT + TEST_SPLIT), random_state=random_state)

os.makedirs(os.path.join(target_directory, dataset_name), exist_ok=True)

train_data.to_csv(os.path.join(target_directory, dataset_name, "train.csv"), index=False)
test_data.to_csv(os.path.join(target_directory, dataset_name, "test.csv"), index=False)
val_data.to_csv(os.path.join(target_directory, dataset_name, "val.csv"), index=False)
size_checker(train_data, test_data, val_data)

The numeber of Train Dataset is: 4857
The numeber of Test Dataset is: 1868
The number of Val dataset is: 748

